# Preamble

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
sys.path.append('../SatLib')
import satPlotTools as spt
import satbox as sb
import orbitalMechanics as om
import utils as utils

In [3]:
import numpy as np
from poliastro import constants
from poliastro.earth import Orbit
from poliastro.earth.sensors import min_and_max_ground_range, ground_range_diff_at_azimuth
from poliastro.bodies import Earth
from poliastro.maneuver import Maneuver
from poliastro.twobody.propagation import propagate
from poliastro.twobody.propagation import cowell
from poliastro.core.perturbations import J2_perturbation
from poliastro.core.propagation import func_twobody
from poliastro.util import norm
import astropy.units as u
from astropy.time import Time, TimeDelta

# Define Walker 

In [4]:
i = 60 * u.deg
t = 4
p = 2
f = 0
alt = 550 * u.km
walker = sb.Constellation.from_walker(i,t,p,f,alt)

/Users/manweichan/Desktop/MITPhD/SAS/code/.venv/lib/python3.7/site-packages/astropy/units/decorators.py:253: UserWarning: Wrapping true anomaly to -π <= nu < π
  return_ = wrapped_function(*func_args, **func_kwargs)


# Define Ground Station

In [5]:
gs_lat = 39 * u.deg
gs_lon = 127 * u.deg
gs_h = 0 * u.m
gs = sb.GroundLoc(gs_lon, gs_lat, gs_h)

# Define RGT Drift Maneuver

In [6]:
r_drift = 650*u.km #Drift orbit at 650 km

# Calculate Burn schedule

In [9]:

schedDict = walker.gen_GOM_2_RGT_scheds(r_drift, gs)
sats2Maneuver, driftTimes, sched = walker.get_ascending_descending_per_plane(schedDict) #Assumes one satellite per plane will get there


maneuver schedule taken from satellite object
maneuver schedule taken from satellite object
maneuver schedule taken from satellite object
maneuver schedule taken from satellite object


# Propagate satellites

In [12]:
t2propagate = 1*u.day
tStep = 30 * u.s
verbose = True
walkerSim = sb.SimConstellation(walker, t2propagate, tStep, verbose = verbose)
walkerSim.propagate(select_sched_sats = sats2Maneuver, verbose=verbose)

maneuver schedule taken from satellite object
Not skipping schedule for Plane 0 Sat 0
maneuver schedule taken from satellite object
Not skipping schedule for Plane 0 Sat 1
maneuver schedule taken from satellite object
Not skipping schedule for Plane 1 Sat 2
maneuver schedule taken from satellite object
Not skipping schedule for Plane 1 Sat 3


In [16]:
#Here is the consetllation you want to plot @jace
walkerSim.constellation